#10. The pyomo Command

The <span style="color:darkblue; font-family:Courier">pyomo</span> command is issued to the DOS prompt or a Unix shell. To see a list of Pyomo command line options, use:



In [1]:
!pyomo solve --help

usage: pyomo solve [options] <model_or_config_file> [<data_files>]

positional arguments:
  model_or_config_file  A Python module that defines a Pyomo model, or a
                        configuration file that defines options for 'pyomo
                        solve' (in either YAML or JSON format)
  data_files            Pyomo data files that defined data used to initialize
                        the model (specified in the first argument)

optional arguments:
  -h, --help            show this help message and exit
  --generate-config-template TEMPLATE
                        Create a configuration template file in YAML or JSON
                        and exit.
  --solver SOLVER       Solver name. This option is required unless the solver
                        name is specified in a configuration file.

Description:

The 'pyomo solve' subcommand optimizes a Pyomo model.  The --solver option
is required because the specific steps executed are solver dependent.
The standard steps ex


>#####Note
>There are two dashes before <span style="color:darkblue; font-family:Courier">help</span>.
In this section we will detail some of the options.

###10.1. Passing Options to a Solver
To pass arguments to a solver when using the <span style="color:darkblue; font-family:Courier">pyomo solve</span> command, appned the Pyomo command line with the argument <span style="color:darkblue; font-family:Courier">--solver-options=</span> followed by an argument that is a string to be sent to the solver (perhaps with dashes added by Pyomo). So for most MIP solvers, the mip gap can be set using



In [2]:
--solver-options= "mipgap=0.01 "

SyntaxError: can't assign to operator (<ipython-input-2-508e25e208c7>, line 1)

Multiple options are separated by a space. Options that do not take an argument should be specified with the equals sign followed by either a space or the end of the string.

For example, to specify that the solver is GLPK, then to specify a mipgap of two percent and the GLPK cuts option, use

In [3]:
--solver=glpk --solver-options="mipgap=0.02 cuts="

SyntaxError: can't assign to operator (<ipython-input-3-73b64f9420c2>, line 1)

If there are multiple "levels" to the keyword, as is the case for some Gurobi and CPLEX options, the tokens are separated by underscore. For example, <span style="color:darkblue; font-family:Courier">mip cuts all</span> would be specified as <span style="color:darkblue; font-family:Courier">mip_cuts_all</span>. For another example, to set the solver to be CPLEX, then to set a mip gap of one percent and to specify <span style="color:darkblue; font-family:Courier">y</span> for the sub-option <span style="color:darkblue; font-family:Courier">numerical</span> to the option <span style="color:darkblue; font-family:Courier">emphasis</span> use

In [4]:
--solver=cplex --solver-options="mipgap=0.001 emphasis_numerical=y"

SyntaxError: can't assign to operator (<ipython-input-4-f506fdfc13f5>, line 1)

See [SolverOpts](https://software.sandia.gov/downloads/pub/pyomo/PyomoOnlineDocs.html#SolverOpts) for a discusion of passing options in a script.

##10.2. Troubleshooting
Many of things that can go wrong are covered by error messages, but sometimes they can be confusing or do not provide enough information. Depending on what the troubles are, there might be ways to get a little additional information.

If there are syntax errors in the model file, for example, it can occasionally be helpful to get error messages directly from the Python interpreter rather than through Pyomo. Suppose the name of the model file is scuc.py, then



In [5]:
python scuc.py

SyntaxError: invalid syntax (<ipython-input-5-cb2189737bca>, line 1)

can sometimes give useful information for fixing syntax errors.

When there are no syntax errors, but there troubles reading the data or generating the information to pass to a solver, then the <span style="color:darkblue; font-family:Courier">--verbose</span> option provides a trace of the execution of Pyomo. The user should be aware that for some models this option can generate a lot of output.

If there are troubles with solver (i.e., after Pyomo has output "Applying Solver"), it is often helpful to use the option <span style="color:darkblue; font-family:Courier">--stream-solver</span> that causes the solver output to be displayed rather than trapped. (See [TeeTrue](https://software.sandia.gov/downloads/pub/pyomo/PyomoOnlineDocs.html#TeeTrue) for information about getting this output in a script). Advanced users may wish to examine the files that are generated to be passed to a solver. The type of file generated is controlled by the <span style="color:darkblue; font-family:Courier">--solver-io</span> option and the <span style="color:darkblue; font-family:Courier">--keepfiles</span> option instructs pyomo to keep the files and output their names. However, the <span style="color:darkblue; font-family:Courier">--symbolic-solver-labels</span> option should usually also be specified so that meaningful names are used in these files.

When there seem to be troubles expressing the model, it is often useful to embed print commands in the model in places that will yield helpful information. Consider the following snippet:

In [2]:
def ax_constraint_rule(model, i):
    # return the expression for the constraint for i
    print("ax_constraint_rule was called for i=",i)
    return sum(model.a[i,j] * model.x[j] for j in model.J) >= model.b[i]

# the next line creates one constraint for each member of the set model.I
model.AxbConstraint = Constraint(model.I, rule=ax_constraint_rule)

NameError: name 'Constraint' is not defined

The effect will be to output every member of the set <span style="color:darkblue; font-family:Courier">model.I</span> at the time the constraint named <span style="color:darkblue; font-family:Courier">model.AxbConstraint</span> is constructed.

##10.3. Direct Interfaces to Solvers
In many applications, the default solver interface works well. However, in some cases it is useful to specify the interface using the <span style="color:darkblue; font-family:Courier">solver-io</span> option. For example, if the solver supports a direct Python interface, then the option would be specified on the command line as

In [7]:
--solver-io=python

SyntaxError: can't assign to operator (<ipython-input-7-8c4b1009d6f0>, line 1)

Here are some of the choices:

- lp: generate a standard linear programming format file with filename extension <span style="color:darkblue; font-family:Courier">lp</span>

- nlp: generate a file with a standard format that supports linear and nonlinear optimization with filename extension <span style="color:darkblue; font-family:Courier">n1lp</span>

- os: generate an OSiL format XML file.

- python: use the direct Python interface.

Note that not all solvers support all interfaces.

